In [31]:
import os, json, ast
import openai
import pandas as pd

In [32]:
# Read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [33]:
## Initialize the conversation with OpenAI
# The instructions provide the following information using chain of thoughts and few shot prompting
# The basic information required for booking the hotel
# Instructions on how to handle user prompts
# An example of how to present an itinerary
# 3 examples of few shot prompting

def initialize_conversation():
    delimiter = "####"
    
    example_recommend_dict = {'Name': "_",
                              'Rate per night': "_",
                              'Area': "_",
                              'Room type': "_",
                              'Total Hotel Price': "_"}
    
    example_itinerary_dict = {'Day 1': "_",
                              'Day 2': "_",
                              'Day 3': "_",
                              'Day N': "_"}
    
    system_message = f"""
    
    You are a travel planner and your goal is to find the suitable hotel for a user.
    
    {delimiter}
    Here are some instructions around the user requirements
    To find the perfect hotel, ask for more information 
    1. Destination city or area
    2. Check-in and check-out dates
    3. Number of guests (adults and children)
    4. Room preferences (e.g. single room, double room, suite)
    5. Specific amenities you are looking for (e.g. pool, gym, free breakfast)
    6. Business or pleasure travel
    7. Budget per night
    {delimiter}
    
    {delimiter}
    Here is a suggested chain of thought.
    Ask a question to understand the user's profile and requirements. 
    If their answers are incomplete. Ask followup questions to understand their needs.
    Please ensure you have the answers for all requirements {{'Destination city or area','Check-in date','Check-out date',
    'Number of guests', 'Amenities', 'Budget', 'travel purpose'}}
    If the necessary information has been extracted, only then proceed to the next step.
    Otherwise, rephrase the question to capture their requirements clearly. 
    Once done with hotel recommendations, if the trip is for pleasure, suggest the itinerary for the duration between check-in date and check-out date
    Itinerary includes popular tourist places where the user can visit each day.
    If the user asks for multiple cities, address the cities one by one.
    {delimiter}
    
    {delimiter}
    Here is how you can present a sample itinerary if the user requests. This is a sample. Please prepare itinerary as per the destination 
    Day 1: Arrive at New York and check-in to Hotel
           Visit Lower Manhattan 
           Explore Battery Park and take a sunset stroll along the Hudson River
           
    Day 2: Visit the Empire State Building
           Visit the bustling Times Square
           Experience Broadway
           
    Day 3: Take the United Nation tour
           Stroll through the Upper West Side
           Enjoy dinner at a cozy restaurant in Upper Manhattan
    
    Day 4: Explore Brooklyn and cross the Brooklyn Bridge
           Checkout from the hotel and depart New York city

    {delimiter}
    
    {delimiter}
    Here is a sample conversation between the user and assistant:
    Assistant: "Hi, I am your travel planner. How can I help you today?"
    User: "Hi, I am looking for a hotel."
    Assistant: "I am glad to find you the perfect hotel. Please provide me the following information. Please provide me 
    destination city or area, check-in and check-out date, number of guest (adult and children), Room preferences (single room, 
    double room, suite), specific amenities (pool, gym, breakfast), budget per night
    User: "My city is Paris. I am checking in on 10th July 2024 and checking out on 14th July 2024. I prefer single room with breakfast.
    My max budget is $100 per night"
    Assistant: "Thank you for providing that information. Are you travelling for business or pleasure?
    User: "I am travelling for business"
    Assistant: "Thank you for the information. I have one more question. Do you need a hotel near train station?
    User: "Not Required"
    Assistant:"Here are top 3 hotel options for you.
    Assistant: "{example_recommend_dict}"
    {delimiter}
    
    {delimiter}
    Here is a sample conversation between the user and assistant:
    Assistant: "Hi, I am your travel planner. How can I help you today?"
    User: "Hi, I am looking for a hotel."
    Assistant: "I am glad to find you the perfect hotel. Please provide me the following information. Please provide me 
    destination city or area, check-in and check-out date, number of guest (adult and children), Room preferences (single room, 
    double room, suite), specific amenities (pool, gym, breakfast), budget per night
    User: "I want to buy a laptop"
    Assistant: "I understand but I am tasked to help you book a hotel"
    User: "I am hungry"
    Assistant: "I understand but I am tasked to help you book a hotel"
    User: "I want to dance"
    Assistant: "I understand but I am tasked to help you book a hotel"
    User: "I want to visit Jaipur from 10th Sept to 14th Sept with 2 adults and 2 children. I would prefer a premium room with a budget of Rs. 8000 per night"
    Assistant: "Thank you for providing that information. Are you travelling for business or pleasure?
    User: "I am travelling for pleasure with family and kids"
    Assistant: "That is great. Let me check the hotels in Jaipur for you"
    Assistant:"Here are top 3 hotel options for you.
    Assistant: "{example_recommend_dict}"
    Assistant: "Since you are travelling for pleasure, would you want me to recommend itinerary. Please say itinerary."
    User: "Yes, please"
    Assistant: "Here are some popular tourist places that you can visit day wise"
    Assistant: "{example_itinerary_dict}"
    {delimiter}
    
    {delimiter}
    Here is a sample conversation between the user and assistant for multiple cities:
    Assistant: "Hi, I am your travel planner. How can I help you today?"
    User: "Hi, I am looking for a hotel."
    Assistant: "I am glad to find you the perfect hotel. Please provide me the following information. Please provide me 
    destination city or area, check-in and check-out date, number of guest (adult and children), Room preferences (single room, 
    double room, suite), specific amenities (pool, gym, breakfast), budget per night
    User: "I want to visit Denver from 10th Sept to 14th Sept and Los Angeles from 14th Sept to 18th Sept with 2 adults and 2 children. Please suggest budget friendly hotel with less than $100 per night.
    Assistant: "Thank you for providing that information. Are you travelling for business or pleasure?
    User: "I am travelling for pleasure with family and kids"
    Assistant: "That is great. Let me check the hotels in Denver for you first"
    Assistant:"Here are top 3 hotel options for you for Denver."
    Assistant: "{example_recommend_dict}"
    User: thanks 
    Assistant: "Here are top 3 hotel options for you for Los Angeles."
    Assistant: "{example_recommend_dict}"
    Assistant: "Since you are travelling for pleasure, would you want me to recommend itinerary. Please say itinerary."
    User: "itinerary"
    Assistant: "Here are some popular tourist places that you can visit day wise in Denver"
    Assistant: "{example_itinerary_dict}"
    User: Thanks
    Assistant: "Here are some popular tourist places that you can visit day wise in Los Angeles"
    Assistant: "{example_itinerary_dict}"
    {delimiter}
       
    """    
    conversation = [{"role": "system", "content": system_message}]
    
    return conversation

In [34]:
# Function as provided by the instructor

def get_chat_completions(input):
    MODEL = 'gpt-3.5-turbo'

    chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

    output = chat_completion.choices[0].message.content

    return output

In [35]:
# Function as provided by the instructor

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [36]:
# Define the structure of the custom function
# Ideally the information of this function will be provided to api

custom_functions = [{
    "name": "findBestHotels",
    "description": "Find the best 3 hotels for a given destination",
    "parameters": {
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "The desired travel destination"
            },
            
            "check-in date": {
                "type" : "string",
                "description": "The check-in date"
            },
            
            "check-out date": {
                "type" : "string",
                "description": "The check-out date"
            },
            
            "number of guests": {
                "type" : "integer",
                "description": "The number of adults and children"
            },
            
            "room prefernce": {
                "type" : "string",
                "description" : "Desired type of room"
            },
            
            "amenities" : {
                "type" : "string",
                "description" : "Desired Amenities"
            },
            
            "budget" : {
                "type" : "integer",
                "description" : "Budget per night"
            }
        }
    }
}]

In [37]:
# Function as provided by the instructor
# Some modifications have been made on the prompt specificaly around the information that is required.

def intent_confirmation_layer(response_assistant):
    prompt = f"""
    
    The goal is to determine whether all the inputs are available to suggest hotel recommendations
    You are provided an input. You need to evaluate if the input text has all the following values:
    Destination city or area
    Check-in and check-out dates
    Room preferences
    Amenities
    Budget per night
    Number of guests
    Business or Pleasure Travel
    
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' only if you have all the information for making hotel recommendations like destination, check-in date, check-out date, room type, amenities, budget per night, guests, 
    Thought 2 - If information is missing output 'No'.
    Thought 2 - Think carefully before the answering.
    """
    
    messages=[{"role": "system", "content": prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    )

    json_output = json.loads(response.choices[0].message.content)
    
    return json_output



In [38]:
# call the OpenAI function using the option of functions

def get_chat_completions_with_function_call(input):
    MODEL = 'gpt-3.5-turbo'

    response = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            functions = custom_functions,
            function_call = 'auto',
            seed = 2345)

    output = json.loads(response.choices[0].message.function_call.arguments)

    return output

In [39]:
def dialogue_mgmt_system_with_function_call():
    print ("Starting conversation with your Travel Planner...\n")
    # Get the name of the user
    print ("Hi, whom am I talking to?")
    information_complete = False
    user_input = ''
    user_name = ''
    user_name = input("")
    
    print ("\nYour Travel Planner:\nWelcome " + user_name)
    print ("I understand you are looking for hotel. Our automated travel planner will help you. If travel planner is taking time to provide the recommendations, please be patient and provide a reminder periodically.")
    # initialize the conversation
    conversation = initialize_conversation()
    
    introduction = get_chat_completions(conversation)
    
    # Initiate the conversation by sending a message that the user is looking for a hotel
    conversation.append({"role": "user", "content": "Hi, I am looking for a hotel."})
    response_assistant = get_chat_completions(conversation)
    print ("\nYour Travel Planner: ")
    print (response_assistant)
    
    while(user_input != "exit"):
       
        print ("\n" + user_name + ":")
        user_input = input("")
        if (user_input == "exit"):
            break

        # Check if content if flagged
        moderation = moderation_check(user_input)

        if (moderation == 'Flagged'):
            print ("Your Travel Planner: " + "I specialize in hotel booking")
        else:
            conversation.append({"role": "user", "content": user_input})
    
        # Provide the user input with history to OpenAI
        response_assistant = get_chat_completions(conversation)  
        
        # check if all the information has been received.
        confirmation = intent_confirmation_layer(conversation)
        if (information_complete == False): # use this flag to avoid entering this function more than once once all information is retrieved
            if "Yes" in confirmation.get('result'):
                information_complete = True
                # this is just for experimenting. There is no API call but ideally an API call could be made to the database
                # JSON structure with correct information is generated
                json_structure = get_chat_completions_with_function_call(conversation)
                print ("\nSystem: Thanks for the information. Here is what you requested")
                print (json_structure)
       
        print ("\nYour Travel Planner: ")
        print (response_assistant)

In [40]:
# If the travel planner is taking time to provide the recommendations, please provide a prompt like "Any update?"
# If the travel planner asks for itinerary, please type the word itinerary

dialogue_mgmt_system_with_function_call()

Starting conversation with your Travel Planner...

Hi, whom am I talking to?
Rishi

Your Travel Planner:
Welcome Rishi
I understand you are looking for hotel. Our automated travel planner will help you. If travel planner is taking time to provide the recommendations, please be patient and provide a reminder periodically.

Your Travel Planner: 
I am glad to assist you in finding the perfect hotel. To get started, could you please provide me with the following information:
1. Destination city or area?
2. Check-in and check-out dates?
3. Number of guests (adults and children)?
4. Room preferences (e.g., single room, double room, suite)?
5. Specific amenities you are looking for (e.g., pool, gym, free breakfast)?
6. Travel purpose (business or pleasure)?
7. Budget per night?

Rishi:
Coorg from 11th Sept to 13th Sept

Your Travel Planner: 
Great choice! I can help you find the perfect hotel in Coorg. Could you please provide the following information:
1. Number of guests (adults and childre